# Student Demographics of International Schools in Denmark
## How many Danes go to International School?

First, a little throat-clearing. The stats that I used (INST10 on Statbank.dk), aggregate data in a special way. 
Student ancestry is either Danish origin, immigrant, ancestor, and unknown. Immigrants have two foreign parents and were born abroad, ancestors were born in Denmark but their parents are foreigners and Danish children have at least one Danish parent, whether born abroad or not.

So, it is impossible to see from the stats which Danish children have one foreign parent or were born abroad and are now "repats" ... and obviously the repat children who spent some time abroad are invisible in the stats. 

I chose schools with no "national" department, where there are two wings of the school as the data is aggregated across the two departments in such schools.
I chose schools in Jutland and Sjælland. I excluded schools who opened between 2013-2023 because very new schools can be a bit strange statistically speaking.
I chose 2013 to start the data range because there was a huge school reform in the *folkeskoler* sector.

With that said, how many children with Danish origin attend International School?

In [10]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook_connected'  # Set renderer to notebook-connected for Jupyter compatibility



In [11]:
df = pd.read_csv('danish students.csv', encoding='latin-1', header=None)

In [12]:
# Rename the headers, drop the ones I don't need, drop the rows I don't need
headers = ["Measure", "Percentage Danish origin", "School", "Grade"] + [str(year) for year in range(2013, 2024)]
df.columns = headers
df = df.drop(columns=["Measure"])
df = df[df["Percentage Danish origin"] != "Subtotal"]

df = df[~df["School"].isin(["280462 The International School of Billund", "281541 Lolland Internationale Skole"])]
df = df.drop(columns=["Percentage Danish origin"])
df = df.reset_index(drop=True)


In [13]:
#Data Cleaning
# Change . to 0
df.replace(".", 0, inplace=True)
# Convert number strings to floats
# List the columns you want to convert
year_columns = [str(year) for year in range(2013, 2024)]

# Convert only those columns to numeric
df[year_columns] = df[year_columns].apply(pd.to_numeric, errors='coerce')

#Reindex
df = df[~(df[year_columns] == 0).all(axis=1)]


Hello again. Now, if you know what a box plot/whisker plot is, skip this bit—I'm going to explain.

Okay, so if you have a group of schools and want to understand trends in numbers, you might want to know the top, bottom, and middle values. You might also want to know what the average is.

Here's how it works:- You start by finding the 1st quartile (Q1) and 3rd quartile (Q3). This means that 25% of schools are below Q1, and 25% are above Q3. You draw a box from Q1 to Q3, which shows the middle 50% of values (the interquartile range, IQR).

Then, you mark a line at the median (the middle value). The “whiskers” usually extend from the box up to 1.5 times the "interquartile range" from Q1 and Q3. Any values outside this range are outliers, which you can plot as individual points.

Optionally, a dotted line for the mean can be added, showing the average.

Now, you can see at a glance how the schools compare and observe any trends over the years. Ready? ...

In [14]:
melted_df = df.melt(id_vars=["School", "Grade"], value_vars=[str(year) for year in range(2013, 2024)], 
                    var_name="Year", value_name="Percentage")
melted_df["Year"] = melted_df["Year"].astype(int)

# Create subplots, 2x2 layout
fig = make_subplots(rows=2, cols=2, subplot_titles=("1.-3. grade", "4.- 6. grade", "7.-9. grade"))

# Define grade levels and subplot positions
grades = [ "Indskoling", "Mellemtrin", "Udskoling"]
positions = [(1, 1), (1, 2), (2, 1)]
colors = ["#C39BD3", "#9B59B6", "#5e3370"]

# Add a box plot for each grade level in its respective subplot
for grade, pos, color in zip(grades, positions, colors):
    grade_data = melted_df[melted_df["Grade"] == grade]
    fig.add_trace(
        go.Box(
            x=grade_data["Year"], 
            y=grade_data["Percentage"], 
            name=grade, 
            boxmean=True,
            marker_color=color  # Apply the specific color for each grade level
        ),
        row=pos[0], col=pos[1]
    )

# Update layout for readability
fig.update_layout(
    title="Percentage of Danish Children by Grade and Year Across Schools",
    height=700,
    width=900,
    showlegend=False  
)

# Update axes labels
fig.update_xaxes(title_text="Year", tickmode="linear")
fig.update_yaxes(title_text="Percentage of Danish Children", range=[0, 100])

fig.show()


We can now see that there are three trends for the three year groups. (I dropped 0th grade to try to show the overall trends without the statistically weirder group obscuring patterns)

Indskoling (1st-3rd Grade):

The average percentage of Danish-origin students indeed seems to hover around 30% for most years.
There was more variation in the earlier years.
The slight decline in recent years suggests that Indskoling is becoming "slightly less popular" among Danish-origin students.

Mellemtrin (4th-6th Grade):

The average percentage of Danish-origin students has varied between approximately 20% and 40%, with some oscillation year-to-year.
The overall trend shows a decrease in variation, indicating a more consistent proportion of Danish students across schools in the middle grades over time.
This stability suggests that, while there's some fluctuation, the range is narrowing, and fewer schools have outlier levels of Danish-origin students in Mellemtrin.

Udskoling (7th-9th Grade):

The trend shows a gradual increase in the average percentage of Danish-origin students, moving from around 30% up to 40%.
Variation among schools has decreased over time, but some schools still show up to 70% Danish-origin students, indicating that these grades are seeing more Danish students in certain cases.
The decreasing variation, along with the increasing average, aligns with a growing interest among Danish students in the oldest grade levels.

In [15]:
# Filter the data for the specified school and exclude "0th Grade"
school_data = df[(df["School"] == "280397 Aarhus Academy for Global Education") & (df["Grade"] != "0th Grade")]

# Melt the DataFrame for plotting
melted_school_data = school_data.melt(id_vars=["School", "Grade"], value_vars=[str(year) for year in range(2013, 2024)],
                                      var_name="Year", value_name="Percentage")
melted_school_data["Year"] = melted_school_data["Year"].astype(int)  # Convert years to integer for proper ordering

# Define a custom color palette for the grades
custom_colors = {
    "Indskoling": "#C39BD3",   # Light purple
    "Mellemtrin": "#9B59B6",   # Slightly darker purple
    "Udskoling": "#5e3370"     # Deep purple
}

# Create the grouped bar chart with custom colors
fig = px.bar(melted_school_data, x="Year", y="Percentage", color="Grade", barmode="group",
             color_discrete_map=custom_colors,
             title="Percentage of Danish Children at AIS",
             labels={"Percentage": "Percentage of Danish Children", "Year": "Year"},
             range_y=[0, 100])  # Set Y-axis range to 0-100%

# Display the chart
fig.show()


Let's look at the historical data from one of my old schools because I know a few of the circumstances that might explain the variation in numbers

You can see that in the youngest grades, 30-40% of students were Danish origin. The numbers dip at 2018-2023 because a bilingual programme opened at another of my old schools which was designed to be attractive to mixed Danish/other nationality families.
The middle grades have not followed the pattern of the younger grades, they got more popular after the competition opened up but then declined presumably as the "missing" children from the younger groups were not being promoted up to those classes.
Udskoling is interesting: Danish origin students were a small minority at 20ish % but were more than half in 2020-21 (because people went home during a pandemic?) but have levelled out to a significant minority of mid 40%.

It will be very interesting to see 2024s data which must be coming out soon... because now the has competition closed,  how many Danish origin children went to AIS in the wake of that decision?

In [16]:
# Filter the data for the specified school and exclude "0th Grade"
school_data = df[(df["School"] == "756002 International School Ikast-Brande") & 
                 (df["Grade"] != "0th Grade") & 
                 (df["Grade"] != "10th Grade")]
# Melt the DataFrame for plotting
melted_school_data = school_data.melt(id_vars=["School", "Grade"], value_vars=[str(year) for year in range(2013, 2024)],
                                      var_name="Year", value_name="Percentage")
melted_school_data["Year"] = melted_school_data["Year"].astype(int)  # Convert years to integer for proper ordering

# Define a custom color palette for the grades
custom_colors = {
    "Indskoling": "#C39BD3",   # Light purple
    "Mellemtrin": "#9B59B6",   # Slightly darker purple
    "Udskoling": "#5e3370"     # Deep purple
}

# Create the grouped bar chart with custom colors
fig = px.bar(melted_school_data, x="Year", y="Percentage", color="Grade", barmode="group",
             color_discrete_map=custom_colors,
             title="Percentage of Danish Children at ISIB",
             labels={"Percentage": "Percentage of Danish Children", "Year": "Year"},
             range_y=[0, 100])  # Set Y-axis range to 0-100%

# Display the chart
fig.show()


I don't know heaps about Ikast-Brande International School but these numbers are extraordinary. 
More than half of their students in the oldest grades are Danish origin and that was historically true for a while in the middle grades. The proportion of Danish  students in the younger years have been reasonably steady around 30%

In [17]:

# Filter the data for the specified school and exclude "0th Grade"
school_data = df[(df["School"] == "280205 Esbjerg International School") & 
                 (df["Grade"] != "0th Grade") & 
                 (df["Grade"] != "10th Grade")]
# Melt the DataFrame for plotting
melted_school_data = school_data.melt(id_vars=["School", "Grade"], value_vars=[str(year) for year in range(2013, 2024)],
                                      var_name="Year", value_name="Percentage")
melted_school_data["Year"] = melted_school_data["Year"].astype(int)  # Convert years to integer for proper ordering

# Define a custom color palette for the grades
custom_colors = {
    "Indskoling": "#C39BD3",   # Light purple
    "Mellemtrin": "#9B59B6",   # Slightly darker purple
    "Udskoling": "#5e3370"     # Deep purple
}

# Create the grouped bar chart with custom colors
fig = px.bar(melted_school_data, x="Year", y="Percentage", color="Grade", barmode="group",
             color_discrete_map=custom_colors,
             title="Percentage of Danish Children at EIS",
             labels={"Percentage": "Percentage of Danish Children", "Year": "Year"},
             range_y=[0, 100])  # Set Y-axis range to 0-100%

# Display the chart
fig.show()



Again, I don't know that much about Esbjerg International School but the oldest grade's trend is similar to ISIB's and the trend for the middle grades is almost a copy past of their oldest's grades. There were more Danish origin students in the youngest classes but this has been gradually declining.

# Who are the Danish children?

Are the Danish origin children we are talking about mixed heritage? Are they repats or are their parents thinking about taking a job abroad? Or are they Danish kids who just fancy the academic challenge of having their subjects in a foreign language?

# So what?

There are two issues to consider when answering this question. 
1) How does this affect the students themselves? (bonus sub question, what about society?)
2) How does this affect the school?

# The Students
There are many benefits to a bilingual education, including for children who are just looking for an extra challenge or who simply like the English language. However, international schools are a small parallel community for foreigners who do not intend to stay in Denmark. Danish schools prepare their students for living in Denmark and give them a grounding in cultural values and norms. Danish children who attend international schools are being educated as Third Culture Kids. 
This could be the best thing that ever happened to Denmark: shaking things up and getting the next generation to be internationally-minded. Or it could cause difficulties for Danish children who don't fit in at home. 

Many of the Danish children attracted to International Schools are academic high fliers BUT they have had one fewer year of school, their content base is not as broad to draw from and they are working in their second language for the first time so need to pick up academic and subject specific vocabulary.

Some of the Danish children are attracted to International schools because they are not thriving in their ordinary school. This might be, as said above, due to not being challenged (This might be why the turning point for attracting Danish origin children is often in the oldest years), but it also could be due to being neurodiverse and finding big schools, with large classes unpleasant. 
Many international schools are smaller and have fewer students in their classes, allowing for more individual attention. And... Neurodiverse Danish students often are experts in English as they follow their special interests on YouTube and most of the content they have watched is in English.

# The International Schools

Many international schools market themselves as being especially academically rigorous to attract families with a strong tradition for school achievement. The grades they receive in exams are important to the school's board and are advertised on the website. Many international school boards desire above average global rankings.

Having a large proportion of Danish students presents a special challenge. Particularly, if the reason they are there is because of unsupported special educational needs (diagnosed or otherwise). This transforms the dynamic in International Schools into something a bit different than the original plan. Of course, many students with international backgrounds have special needs but the balance is shifted by having so many Danish students who are there because they couldn't thrive in their local school. It is also particularly difficult to get students through exams in the final three years of an 11 year programme, which is when International Schools become more popular for Danish origin students.

This put International Schools in a difficult position: Are they globally elite academic institutions or are they a final resort for Danish children with nowhere else to go?

# Conclusion

The trend I have identified has existed for some time. Danish children are increasingly attending international schools— whether it's for academic challenge, a chance to study in a foreign language, or the need for a smaller, more supportive educational environment.

This shift presents both opportunities and challenges. On one hand, it could encourage a new generation of Danish students to become globally minded, culturally flexible, and linguistically adept—skills that are highly valuable in an interconnected world. These students, raised in bilingual or multilingual settings, might bring fresh perspectives and adaptability that benefit Danish society as a whole.

On the other hand, it creates a delicate balance for the schools themselves. As they accommodate more Danish students, some with challenging educational needs, they risk shifting from their original mission of serving globally mobile families. If too many students join international schools as a “last resort,” this could alter the academic rigor and international focus that these institutions are known for. The increasing presence of Danish students could necessitate additional resources and adjustments, particularly if some are there due to unmet needs in the Danish school system.

Ultimately, this trend reflects a complex intersection between Danish society and the evolving role of international schools. If thoughtfully addressed, this could lead to a richer, more inclusive educational landscape. However, without careful consideration, international schools may face tension between maintaining their elite, globally focused reputation and meeting the growing demands from Danish students who seek an alternative educational path.

In summary, as the Danish student population in international schools grows, these institutions will need to navigate their dual identity: Are they academically rigorous spaces for global citizens, or are they becoming a vital alternative for Danish students in need of a different educational experience?

